In [40]:
import rdata
import pandas as pd
import numpy as np
from libpysal.weights import full2W
from spreg import ML_Lag
import networkx as nx


def preprocess(adjacency_matrix):
    np.random.seed(1126)
    N = adjacency_matrix.shape[0]

    non_zero = np.argwhere(adjacency_matrix.diagonal() == 1)
    if len(non_zero) != 0:
        for i in non_zero[0]:
            adjacency_matrix[i][i] = 0

    # iso = np.where(~adjacency_matrix.any(axis=1))[0]
    # for i in iso:
    #     idx = np.random.choice(range(N))
    #     while idx == i:
    #         idx = np.random.choice(range(N))
    #     adjacency_matrix[i][idx] = 1


data_matrix, adjacency_matrix = rdata.read_rda(
    'dataset_network_interactions.RData'
)['dataset_lecture3']
preprocess(adjacency_matrix)
W = full2W(adjacency_matrix)

/home/r12323011/.local/lib/python3.10/site-packages/libpysal/weights/util.py:768: UserWarning: The weights matrix is not fully connected: 
 There are 132 disconnected components.
 There are 93 islands with ids: 3, 8, 25, 30, 32, 33, 35, 48, 53, 60, 61, 68, 72, 74, 76, 80, 82, 83, 87, 88, 94, 95, 99, 102, 107, 110, 113, 116, 117, 125, 127, 130, 131, 134, 136, 137, 140, 141, 150, 152, 157, 158, 168, 169, 170, 174, 177, 180, 181, 183, 192, 197, 199, 202, 208, 212, 218, 220, 222, 230, 241, 243, 245, 247, 248, 250, 251, 252, 254, 256, 258, 266, 268, 271, 273, 274, 276, 279, 283, 284, 285, 288, 291, 292, 295, 296, 300, 308, 314, 316, 317, 329, 333.
  return W(neighbors, weights, id_order=ids, **kwargs)


In [41]:
_X = pd.DataFrame(
    data_matrix,
    columns=[
        'gpa', 'smoke_freq', 'drink', 'club', 'exer',
        'male', 'black', 'hisp', 'asian', 'race_other',
        'both_par', 'less_hs', 'more_hs', 'momedu_miss',
        'Prof', 'Home', 'job_other'
    ]
)
y = _X.gpa.to_numpy()
name_x = [
    'male', 'black', 'hisp', 'asian',
    'race_other', 'both_par', 'less_hs',
    'more_hs', 'momedu_miss', 'Prof',
    'Home', 'job_other'
]
X = _X[name_x]

## Problem 1 
Using the data set dataset_network_interactions provided in the course demonstration, estimate the spatial Durbin model where the regressors WX are included. Report your estimation results.

In [43]:
W.transform = 'r'
WX = np.dot(W.full()[0], X)
X_extended = np.hstack([X, WX])

model = ML_Lag(
    y, X_extended, W,
    method='full',
    name_y = 'gpa',
    name_x = name_x + [f'spatial_{i}'for i in name_x]
)
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         gpa                Number of Observations:         336
Mean dependent var  :      2.9779                Number of Variables   :          26
S.D. dependent var  :      0.6925                Degrees of Freedom    :         310
Pseudo R-squared    :      0.2196
Spatial Pseudo R-squared:  0.2096
Log likelihood      :   -311.7780
Sigma-square ML     :      0.3732                Akaike info criterion :     675.556
S.E of regression   :      0.6109                Schwarz criterion     :     774.801

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

## Problem 2
Change the specification of weights matrix to binary values, i.e., do not row-standardize the weights matrix, and estimate the above spatial Durbin model again. Report your estimation results. 

In [47]:
W.transform = 'b'
WX = np.dot(W.full()[0], X)
X_extended = np.hstack([X, WX])

model = ML_Lag(
    y, X_extended, W,
    method='full',
    name_y = 'gpa',
    name_x = name_x + [f'spatial_{i}'for i in name_x]
)
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         gpa                Number of Observations:         336
Mean dependent var  :      2.9779                Number of Variables   :          26
S.D. dependent var  :      0.6925                Degrees of Freedom    :         310
Pseudo R-squared    :      0.1854
Spatial Pseudo R-squared:  0.1773
Log likelihood      :   -318.6715
Sigma-square ML     :      0.3895                Akaike info criterion :     689.343
S.E of regression   :      0.6241                Schwarz criterion     :     788.588

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

/home/r12323011/.local/lib/python3.10/site-packages/spreg/ml_lag.py:634: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


## Problem 3
Redo 2. by adding an additional regressor of individual network degree. Report your estimation results. 

In [28]:
G = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)
in_degree_centrality = nx.in_degree_centrality(G)
out_degree_centrality = nx.out_degree_centrality(G)

in_degree = np.array(list(in_degree_centrality.values()))
out_degree = np.array(list(out_degree_centrality.values()))
degree = (in_degree + out_degree).reshape(-1, 1)

In [29]:
X_extended = np.hstack([degree, X, WX])

model = ML_Lag(
    y, X_extended, W,
    method='full',
    name_y = 'gpa',
    name_x = ['total_degree'] + name_x + [f'spatial_{i}'for i in name_x]
)
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         gpa                Number of Observations:         336
Mean dependent var  :      2.9779                Number of Variables   :          27
S.D. dependent var  :      0.6925                Degrees of Freedom    :         309
Pseudo R-squared    :      0.1944
Spatial Pseudo R-squared:  0.1802
Log likelihood      :   -317.5684
Sigma-square ML     :      0.3853                Akaike info criterion :     689.137
S.E of regression   :      0.6207                Schwarz criterion     :     792.199

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

/home/r12323011/.local/lib/python3.10/site-packages/spreg/ml_lag.py:634: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))


### contextualized degree centrality

In [39]:
X_degree = pd.concat([pd.DataFrame(degree), X], axis=1).rename(columns={0: 'degree'})
WX = np.dot(W.full()[0], X_degree)
X_extended = np.hstack([X_degree, WX])

model = ML_Lag(
    y, X_extended, W,
    method='full',
    name_y = 'gpa',
    name_x = ['total_degree'] + name_x + ['spatial_total_degree'] + [f'spatial_{i}'for i in name_x]
)
print(model.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :         gpa                Number of Observations:         336
Mean dependent var  :      2.9779                Number of Variables   :          28
S.D. dependent var  :      0.6925                Degrees of Freedom    :         308
Pseudo R-squared    :      0.1945
Spatial Pseudo R-squared:  0.1808
Log likelihood      :   -317.5531
Sigma-square ML     :      0.3852                Akaike info criterion :     691.106
S.E of regression   :      0.6207                Schwarz criterion     :     797.985

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

/home/r12323011/.local/lib/python3.10/site-packages/spreg/ml_lag.py:634: RuntimeWarning: invalid value encountered in log
  jacob = np.log(np.linalg.det(a))
